In [134]:
# Carbon Calculator

In [65]:
import pandas as pd
import numpy as np

In [5]:
# Confirm the cement type
cement_type = input("Enter the cement type: ")

# Pre-set: Calculate limestone required for 1 tonne cement
cement_unit_mass = 1

Enter the cement type:  opc


In [7]:
# a general formula to calculate emissions from emission factors of fossil fuel or electricity energy mix
def combustion_emission (fuel, emission_factor):
    return fuel * emission_factor

def power_emission(energy_mix, emission_energy_mix):
    return energy_mix * emission_energy_mix

In [9]:
# A1 emission: CO2 emission from raw material supply
# A2 emission: CO2 from transport
# A3 emission: CO2 from manufacturing

# A1 stage contains 1) Quarrying raw materials, 2) Crushing, 3) Preparing raw meal
# A2 stage transport
# A3 stage contains 1) fuel combustion, 2) chemical reaction

def a1_emission(extraction, explosives, crushing):
    return extraction + explosives + crushing


In [11]:
# A1.1 Quarrying and A1.2 Crushing

clinker_in_cement = float(input("Enter the clinker in cement (%): "))
# clinker major components
clinker_components = ["tricalcium silicate 3CaO.SiO2", 
                      "dicalcium silicate 2CaO.SiO2", 
                      "tricalcium aluminate 3CaO.Al2O3", 
                      "tetracalcium alumniorferrite 3CaO.Al2O3.Fe2O3", 
                      "Gypsum"
                     ]

# give first type of clinker's components weights
type_1 = [50,25,10,10,5]
clinker_components = pd.DataFrame({"type 1":type_1}, index=clinker_components)

# give CaO weights in each component
CaO_in_component = [(56*3/228),(56*2/172),(56*3/270),(56*3/429.64),0]
clinker_components.insert(1,"Cao weight",CaO_in_component)
clinker_components["Cao weight"]=clinker_components["Cao weight"].round(2)
print (clinker_components)

# calculate CaO weight in clinker
CaO_in_clinker = (clinker_components["type 1"] * clinker_components["Cao weight"]).sum()
print("CaO weights in clinker {:.2%}.". format(CaO_in_clinker/100))

CaCO3_CaO_ratio = 1.785  # (stoichiometric ratio, as 100/56)
CaCO3_required = cement_unit_mass * clinker_in_cement/100 * CaO_in_clinker/100 * CaCO3_CaO_ratio # unit in tonne
mineral_purity = float(input("Enter the mineral purity (%): "))
limestone_required = float(CaCO3_required / (mineral_purity/100))
print("Limestone required to produce 1 tonne cement: {:.2f} tonne.". format(limestone_required))

Enter the clinker in cement (%):  90


                                               type 1  Cao weight
tricalcium silicate 3CaO.SiO2                      50        0.74
dicalcium silicate 2CaO.SiO2                       25        0.65
tricalcium aluminate 3CaO.Al2O3                    10        0.62
tetracalcium alumniorferrite 3CaO.Al2O3.Fe2O3      10        0.39
Gypsum                                              5        0.00
CaO weights in clinker 63.35%.


Enter the mineral purity (%):  90


Limestone required to produce 1 tonne cement: 1.13 tonne.


In [17]:
# Calculate energy consumed to run crusher 
# Step 1 - jaw crusher - primary crusher used to break large limestone blocks into smaller pieces
# reference jaw crusher is 600t/hr, 220kW 
# https://www.mining-rockcrusher.com/sale-15458633-92000kgs-heavy-duty-jaw-crusher-machine-granite-limestone-crushing.html
crusher_capacity_1 = float(input("Enter the crusher capacity (t/hr): "))
#time_crushing_1_tonne = float(1/crusher_capacity)  # (unit in hrs)
crusher_power_1 = float(input("Enter the crusher power (KW): "))

# Step 2 - Impact crusher / hammer crusher - secondary crushing produces sand-like particles
# reference jaw crusher is 600t/hr, 220kW
crusher_capacity_2 = float(input("Enter the crusher capacity (t/hr): "))
crusher_power_2 = float(input("Enter the crusher power (KW): "))

def energy_crushing (crusher_capacity,crusher_power):
    return (1/crusher_capacity) * crusher_power  # (unit in KWh)

Jaw_crusher_energy = energy_crushing (crusher_capacity_1,crusher_power_1)
hammer_crusher_energy = energy_crushing (crusher_capacity_2,crusher_power_2)
energy_crushing_1_tonne = Jaw_crusher_energy + hammer_crusher_energy

total_energy_crushing = limestone_required * energy_crushing_1_tonne
print("Total energy to crush limestone per 1 tonne cement produced: {:.3f} KWh.".format(total_energy_crushing))

Enter the crusher capacity (t/hr):  600
Enter the crusher power (KW):  220
Enter the crusher capacity (t/hr):  400
Enter the crusher power (KW):  150


Total energy to crush limestone per 1 tonne cement produced: 0.839 KWh.


In [19]:
# define a dataframe for fuel mix in grid network and emission intensity
fuel_mix = pd.DataFrame(columns=["Coal","Natural Gas","Nuclear","Renewables","Other"])

# Input one row representing fuel mix at that location
# UK grid energy sources with CO2 emission from fuel mix 
# UK Reference: https://www.gov.uk/government/publications/fuel-mix-disclosure-data-table/fuel-mix-disclosure-data-table#fn:1
UK_fuel_mix = [5.9, 33.3, 16.2, 42.1, 2.5] # (total 100%)
UK_CO2_intensity_from_fuel_mix = [0.919,0.382,0,0,0.948] # (in kg/KWh)
fuel_mix.loc["UK (%)"] = UK_fuel_mix
fuel_mix.loc["CO2 (kg/KWh)"] = UK_CO2_intensity_from_fuel_mix
print (fuel_mix)

# Average CO2 emission from UK grid mix = Grid mix × CO2 emission from energy mix
UK_average_CO2_fuel_mix = ((fuel_mix.loc["UK (%)"]/100) * fuel_mix.loc["CO2 (kg/KWh)"]).sum()  # CO2 emission per 1KWh energy
print("Average CO2 emission from UK grid: {:.5f} kg CO2eq./KWh.".format(UK_average_CO2_fuel_mix))

               Coal  Natural Gas  Nuclear  Renewables  Other
UK (%)        5.900       33.300     16.2        42.1  2.500
CO2 (kg/KWh)  0.919        0.382      0.0         0.0  0.948
Average CO2 emission from UK grid: 0.20513 kg CO2eq./KWh.


In [21]:
# Calculate CO2 emission for crushing limestone per 1 tonne cement produced

# Calculate CO2 emission from crushing limestone
CO2_emission_crushing_limestone = UK_average_CO2_fuel_mix * total_energy_crushing
print("CO2 emission from crushing limestone per 1 tonne cement produced: {:.2f} kg CO2eq.".format(CO2_emission_crushing_limestone))

CO2 emission from crushing limestone per 1 tonne cement produced: 0.17 kg CO2eq.


In [39]:
# CO2 emission from transport
# Limestone required / Truck loading capacity × Fuel consumption per truck per km × Transporting distance × CO2 emission per 1L diesel
# Truck loading capacity varies case by case (value inputs here) #
truck_loading = 18  # in tonne
fuel_consumption_per_truck_100km = 30 # in l/100km
# Transportation distance from quarrying site to processing unit
distance_1 = 100 # in km, from mine to storage unit
CO2_emission_diesel = 3 # in kg/l
# Calculate the CO2 emission from transportation
CO2_emission_transport_1 = float (limestone_required / truck_loading * (fuel_consumption_per_truck_100km / 100) * distance_1 * CO2_emission_diesel)
print("CO2 emission from transporting limestone per 1 tonne cement produced: {:.2f} kg CO2eq.". format(CO2_emission_transport_1))

CO2 emission from transporting limestone per 1 tonne cement produced: 5.65 kg CO2eq.


In [41]:
# Explosives emission
# Amount of explosive used per 1 tonne limestone extracted × Limestone required for 1 tonne cement × Emission factor of explosive
explosives_amount = float(input("Enter amount of explosives used per 1 tonne limestone extracted (kg): "))
emission_factor_explosives = 0.1
CO2_emission_explosives = float(explosives_amount * limestone_required * emission_factor_explosives)
print("CO2 emission from explosives per 1 tonne cement produced: {:.2f} kg CO2eq.".format(CO2_emission_explosives))

Enter amount of explosives used per 1 tonne limestone extracted (kg):  30


CO2 emission from explosives per 1 tonne cement produced: 3.39 kg CO2eq.


In [45]:
#sum A1 emission
A1_emission = a1_emission(CO2_emission_transport_1,CO2_emission_explosives,CO2_emission_crushing_limestone)
print("A1 emission, CO2 from raw material supply per 1 tonne cement produced: {:.2f} kg CO2eq.".format(A1_emission))

A1 emission, CO2 from raw material supply per 1 tonne cement produced: 9.22 kg CO2eq.


In [53]:

def a3_emission (roller_mill_blender, CO2_process, fuel_combustion, cooling_fan, conveyor):
    return roller_mill_blender + CO2_process + fuel_combustion + cooling_fan + conveyor

In [27]:
# A2 transport

# Limestone required / Truck loading capacity × Fuel consumption per truck per km × Transporting distance × CO2 emission per 1L diesel

# Truck loading capacity 
truck_loading = float(input("truck loading capacity (tonne): "))
fuel_consumption_per_truck_100km = 30 # in l/100km
# Transportation distance from quarrying site to processing unit
distance_2 = 300 # in km, from storage unit to processing plant
CO2_emission_diesel = 3 # in kg/l
# Calculate the CO2 emission from transportation
CO2_emission_transport_2 = float (limestone_required / truck_loading * (fuel_consumption_per_truck_100km / 100) * distance_2 * CO2_emission_diesel)
A2_emission = CO2_emission_transport_2
print("A2 CO2 emission per 1 tonne cement produced: {:.2f} kg CO2eq.". format(A2_emission))

truck loading capacity (tonne):  18


A2 CO2 emission per 1 tonne cement produced: 16.96 kg CO2eq.


In [49]:
# Roller mill blender operation
# Time per blending 1 ton raw materials × raw materials required for 1 ton cement × Energy consumed to run roller mill blender × CO2 emission per 1KWh energy
CO2_roller_mill_blender = 3 # in kg CO2eq

# CO2/CaCO3 Stoichiometric Ratio
CO2_CaCO3_ratio = 0.44
CO2_chemical = CO2_CaCO3_ratio * CaCO3_required * 1000
print("CO2 emission in chemical reaction to produce 1 tonne cement: {:.2f} kg.". format(CO2_chemical))

CO2 emission in chemical reaction to produce 1 tonne cement: 447.80 kg.


In [87]:
# Fuel combustion
# Total heat ≈ sensible heat to raise solids + calcination heat + losses.
# heat requirement in modern dry kilns is typically 3.0–3.4 GJ per tonne of clinker
total_heat = 3000 #unit in MJ
# fuel volume = total heat / fuel lower heating value
natural_gas_LHV = 36.6 # unit in MJ/m3
ng_volume = total_heat / natural_gas_LHV
# ghg conversion factor is used to calculate emissions of different fuels
ng_ghg = 2.06 #unit in kgCO2/m3
CO2_ng = ng_volume * ng_ghg # in kgCO2eq
print ("{:.2f} kg". format (CO2_ng))

#cooling_fan
CO2_cooling_fan = 0 # in kg CO2eq

#conveyor
CO2_conveyor = 0 # in kg CO2eq

A3_emission = a3_emission (CO2_roller_mill_blender, CO2_chemical, CO2_fuel, CO2_cooling_fan, CO2_conveyor)
print("A3 CO2 emission per 1 tonne cement produced: {:.2f} kg CO2eq.". format(A3_emission))

168.85 kg
A3 CO2 emission per 1 tonne cement produced: 750.80 kg CO2eq.


In [89]:
A1_A3_emission = A1_emission + A2_emission + A3_emission

Emission_summary = pd.DataFrame({"Emission (kg CO2)":[A1_emission,A2_emission,A3_emission,A1_A3_emission]}, index=["A1", "A2", "A3", "S1"])
Emission_summary["Emission (kg CO2)"] = Emission_summary["Emission (kg CO2)"].round(2)

Emission_summary["share %"] = Emission_summary["Emission (kg CO2)"]/A1_A3_emission*100
Emission_summary["share %"] = Emission_summary["share %"].round(1)
print(Emission_summary)

    Emission (kg CO2)  share %
A1               9.22      1.2
A2              16.96      2.2
A3             750.80     96.6
S1             776.98    100.0


In [91]:
def carbon_calculator(A1_emission, A2_emission, A3_emission):
    return A1_emission + A2_emission + A3_emission
Total_carbon = carbon_calculator (A1_emission, A2_emission, A3_emission)
print(f"{cement_type} produces {Total_carbon:.2f} kg CO2eq per tonne cement.")

opc produces 776.98 kg CO2eq per tonne cement.
